Task 1 - Employee_data

In [ ]:
import boto3
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Define S3 paths
bucket_name = "poc-bootcamp-capstone-group3"
bronze_prefix = "poc-bootcamp-bronze/employee_data/"
gold_path = f"s3://{bucket_name}/poc-bootcamp-gold/employee_data_output/"
processed_file_key = f"{bronze_prefix}processed_files.txt"

# Expected columns
expected_columns = {"emp_id", "age", "name"}

# PostgreSQL connection details
pg_url = "jdbc:postgresql://54.174.233.120:5432/postgres"
pg_properties = {
    "user": "postgres",
    "password": "11223344",
    "driver": "org.postgresql.Driver"
}
pg_table = "employee"


# Initialize Spark and boto3
spark = SparkSession.builder.appName("DailyEmployeeProcessing").getOrCreate()
s3 = boto3.client('s3')

try:
    # Ensure processed files log exists
    try:
        obj = s3.get_object(Bucket=bucket_name, Key=processed_file_key)
        processed_files = {line.split(',')[0]: float(line.split(',')[1]) for line in obj['Body'].read().decode('utf-8').splitlines() if line}
    except s3.exceptions.NoSuchKey:
        processed_files = {}

    # List all CSV files in bronze
    files = s3.list_objects_v2(Bucket=bucket_name, Prefix=bronze_prefix).get('Contents', [])
    csv_files = [f for f in files if f['Key'].endswith('.csv')]

    # Identify new/updated files
    to_process = []
    for file in csv_files:
        key = file['Key']
        last_modified = file['LastModified'].timestamp()
        if processed_files.get(key) != last_modified:
            to_process.append((key, last_modified))

    if not to_process:
        print("INFO: No new or updated files to process. Exiting gracefully.")
    else:
        for key, last_modified in to_process:
            print(f"Processing file: {key}")
            df = spark.read.option("header", True).option("inferSchema", True).csv(f"s3://{bucket_name}/{key}")

            # Check if all expected columns are present
            actual_columns = set(df.columns)
            missing = expected_columns - actual_columns
            if missing:
                raise ValueError(f"ERROR: Missing expected columns in {key}: {missing}")

            # Clean and validate
            cleaned_df = df.select(
                col("emp_id").cast("string"),
                col("age").cast("int"),
                col("name").cast("string")
            ).dropna().dropDuplicates()

            # Write cleaned data to gold path
            cleaned_df.write.mode("append").parquet(gold_path)

            #Write to Postgres
            cleaned_df.write.mode("append").jdbc(url=pg_url, table=pg_table, properties=pg_properties)

            # Update log
            processed_files[key] = last_modified

        # Write updated log back to S3
        log_content = "\n".join([f"{k},{v}" for k, v in processed_files.items()])
        s3.put_object(Bucket=bucket_name, Key=processed_file_key, Body=log_content)

except Exception as e:
    print(f"ERROR: Job failed with exception: {e}")



Task 2 - leave_quota

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StringType, IntegerType
import boto3

# Define S3 paths
bucket_name = "poc-bootcamp-capstone-group3"
bronze_prefix = "poc-bootcamp-bronze/employee_leave_quota/"
gold_path = f"s3://{bucket_name}/poc-bootcamp-gold/employee_leave_quota_output/"
processed_file_key = f"{bronze_prefix}processed_files.txt"

# PostgreSQL connection details
pg_url = "jdbc:postgresql://54.174.233.120:5432/postgres"
pg_properties = {
    "user": "postgres",
    "password": "11223344",
    "driver": "org.postgresql.Driver"
}
pg_table = "leave_quota"

# Initialize Spark and Boto3
spark = SparkSession.builder.appName("YearlyLeaveQuotaTableJob").getOrCreate()
s3_client = boto3.client('s3')

try:
    # Step 1: Load processed files
    try:
        processed_obj = s3_client.get_object(Bucket=bucket_name, Key=processed_file_key)
        processed_files = processed_obj['Body'].read().decode('utf-8').splitlines()
    except s3_client.exceptions.ClientError:
        processed_files = []

    processed_files_set = set(processed_files) 

    # Step 2: List all files under bronze_prefix
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=bronze_prefix)
    all_files = response.get('Contents', [])

    # Filter only CSV files excluding processed_files.txt itself
    new_files = [obj['Key'] for obj in all_files if obj['Key'].endswith('.csv') and obj['Key'] not in processed_files_set]

    if not new_files:
        print("No new files to process.")
    else:
        for file_key in new_files:
            file_path = f"s3://{bucket_name}/{file_key}"
            file_name = file_key.split("/")[-1]  # Get only filename

            print(f"Processing file: {file_name}")

            # Step 3: Read the file
            df = spark.read.option("header", True).option("inferSchema", True).csv(file_path)

            # Validate required columns
            required_cols = ["emp_id", "leave_quota", "year"]
            if not all(c in df.columns for c in required_cols):
                print(f"Skipping {file_name}: Missing required columns.")
                continue

            # Step 4: Clean and transform
            cleaned_df = (
                df.select(
                    col("emp_id").cast(StringType()),
                    col("leave_quota").cast(IntegerType()),
                    col("year").cast(IntegerType())
                )
                .filter(
                    col("emp_id").isNotNull() &
                    col("leave_quota").isNotNull() &
                    col("year").isNotNull()
                )
                .dropDuplicates(["emp_id", "year"])
            )

            # Step 5: Write to Gold
            print(f"Writing {file_name} data to {gold_path}")
            cleaned_df.write.mode("append").partitionBy("year").parquet(gold_path)

            #Write to Postgres
            cleaned_df.write.mode("append").jdbc(url=pg_url, table=pg_table, properties=pg_properties)

            # Step 6: After successful write, mark as processed
            processed_files_set.add(file_key)

        # Step 7: Update processed_files.txt
        updated_content = "\n".join(processed_files_set)
        s3_client.put_object(Bucket=bucket_name, Key=processed_file_key, Body=updated_content)
        print("Processed files list updated.")

except Exception as e:
    print(f"Job failed: {e}")

Task 3 - leave_calender

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year
from pyspark.sql.types import DateType, StringType
import boto3

# S3 paths
bucket_name = "poc-bootcamp-capstone-group3"
bronze_prefix = "poc-bootcamp-bronze/employee_leave_calendar_data/"
gold_path = f"s3://{bucket_name}/poc-bootcamp-gold/employee_leave_calendar_output/"
processed_file_key = f"{bronze_prefix}processed_files.txt"

# PostgreSQL connection details
pg_url = "jdbc:postgresql://54.174.233.120:5432/postgres"
pg_properties = {
    "user": "postgres",
    "password": "11223344",
    "driver": "org.postgresql.Driver"
}
pg_table = "leave_calendar"

# Initialize Spark session
spark = SparkSession.builder \
    .appName("YearlyLeaveCalendarTableJob") \
    .getOrCreate()

# Initialize boto3 client
s3_client = boto3.client('s3')

try:
    # Step 1: Load processed files
    try:
        processed_obj = s3_client.get_object(Bucket=bucket_name, Key=processed_file_key)
        processed_files = processed_obj['Body'].read().decode('utf-8').splitlines()
    except s3_client.exceptions.ClientError:
        processed_files = []

    processed_files_set = set(processed_files)

    # Step 2: List all files under bronze_prefix
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=bronze_prefix)
    all_files = response.get('Contents', [])

    # Filter only CSV files excluding processed_files.txt itself
    new_files = [obj['Key'] for obj in all_files if obj['Key'].endswith('.csv') and obj['Key'] not in processed_files_set]

    if not new_files:
        print("No new files to process.")
    else:
        for file_key in new_files:
            file_path = f"s3://{bucket_name}/{file_key}"
            file_name = file_key.split("/")[-1]  # Get only filename

            print(f"Processing file: {file_name}")

            # Step 3: Read the file
            df = spark.read.option("header", True).option("inferSchema", True).csv(file_path)

            # Validate required columns
            required_cols = ["date", "reason"]
            if not all(c in df.columns for c in required_cols):
                print(f"Skipping {file_name}: Missing required columns.")
                continue

            # Step 4: Clean and transform
            cleaned_df = (
                df.select(
                    col("date").cast(DateType()),
                    col("reason").cast(StringType())
                )
                .filter(
                    col("date").isNotNull() &
                    col("reason").isNotNull()
                )
                .dropDuplicates(["date", "reason"]) 
            )

            # Add year column
            cleaned_df = cleaned_df.withColumn("year", year(col("date")))

            # Step 5: Write to Gold
            print(f"Writing {file_name} data to {gold_path}")
            cleaned_df.write.mode("append").partitionBy("year").parquet(gold_path)

            #Write to Postgres
            cleaned_df.write.mode("append").jdbc(url=pg_url, table=pg_table, properties=pg_properties)

            # Step 6: After successful write, mark as processed
            processed_files_set.add(file_key)

        # Step 7: Update processed_files.txt
        updated_content = "\n".join(processed_files_set)
        s3_client.put_object(Bucket=bucket_name, Key=processed_file_key, Body=updated_content)
        print("Processed files list updated.")

except Exception as e:
    print(f"Job failed: {e}")

Task 4 - leave_data

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, current_date, to_date,
    sum as sum_, when, lit, year, month, broadcast
)
from pyspark.sql.types import StringType, DateType


BRONZE_PATH = "s3://poc-bootcamp-capstone-group3/poc-bootcamp-bronze/employee_leave_data/"
CONSOLIDATED = "s3://poc-bootcamp-capstone-group3/poc-bootcamp-gold/employee_leave_data_output/"

# PostgreSQL connection details
pg_url = "jdbc:postgresql://54.174.233.120:5432/postgres"
pg_properties = {
    "user": "postgres",
    "password": "11223344",
    "driver": "org.postgresql.Driver"
}
pg_table = "employee_leave_data"

# SparkSession
spark = SparkSession.builder.appName("DailyLeaveSnapshot").getOrCreate()

# 1) LOAD raw events and cast date
raw = (
    spark.read
    .option("header", True)
    .csv(BRONZE_PATH)
    .withColumn("date", to_date(col("date"), "yyyy-M-d"))
)

# 2) AGGREGATE to find dominant status per emp/date
raw_agg = (
    raw.groupBy("emp_id", "date")
    .agg(
        sum_(when(col("status") == "ACTIVE", 1).otherwise(0)).alias("cnt_active"),
        sum_(when(col("status") == "CANCELLED", 1).otherwise(0)).alias("cnt_cancelled")
    )
    .withColumn(
        "new_status",
        when(col("cnt_active") > col("cnt_cancelled"), lit("ACTIVE"))
        .otherwise(lit("CANCELLED"))  # includes tie and CANCELLED majority cases
    )
    .select("emp_id", "date", "new_status")
)

# 3) READ previous snapshot (if any)
try:
    hist = (
        spark.read
        .parquet(CONSOLIDATED)
        .select("emp_id", "date", "status", "ingestion_date")
    )
except Exception as e:
    # If no history exists, create an empty DataFrame with the required schema.
    hist = spark.createDataFrame(
        [],
        schema=raw_agg.schema.add("status", StringType()).add("ingestion_date", DateType())
    )

# 4) MERGE logic: outer-join of aggregated (raw_agg) & history
merged = hist.alias("h").join(
    raw_agg.alias("r"),
    on=["emp_id", "date"],
    how="outer"
)

# 5) Decide final status and ingestion_date
today = current_date()
result = (
    merged.withColumn(
        "final_status",
        when(col("r.new_status").isNotNull(), col("r.new_status"))
        .otherwise(col("h.status"))
    )
    .withColumn(
        "ingestion_date",
        when(col("r.new_status").isNotNull(), today)
        .otherwise(col("h.ingestion_date"))
    )
    .filter(col("final_status").isNotNull())
    .select(
        col("emp_id"),
        col("date"),
        col("final_status").alias("status"),
        col("ingestion_date")
    )
)

# 6) Add partitions based on the leave date
result_with_partition = (
    result
    .withColumn("year", year(col("date")))
    .withColumn("month", month(col("date")))
)


# Broadcast join with Employee Timeframe data
employee_timeframe_path = "s3://poc-bootcamp-capstone-group3/poc-bootcamp-gold/employee_timeframe_data_output/"

# Load the employee timeframe data and filter to get only active employees.
employee_active = (
    spark.read.parquet(employee_timeframe_path)
    .filter(col("status") == "ACTIVE")
    .select("emp_id")
    .distinct()
    .withColumnRenamed("emp_id", "active_emp_id")
)

# Perform a broadcast join to check if each employee in the leave data is active.
# Then, update the leave status:
# - If the employee is NOT active (no match found), force the status to "CANCELLED".
# - Otherwise, retain the existing leave status from the aggregation logic.
result_with_partition = result_with_partition.join(from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, current_date, to_date,
    sum as sum_, when, lit, year, month, broadcast
)
from pyspark.sql.types import StringType, DateType


BRONZE_PATH = "s3://poc-bootcamp-capstone-group3/poc-bootcamp-bronze/employee_leave_data/"
CONSOLIDATED = "s3://poc-bootcamp-capstone-group3/poc-bootcamp-gold/employee_leave_data_output/"

# PostgreSQL connection details
pg_url = "jdbc:postgresql://54.174.233.120:5432/postgres"
pg_properties = {
    "user": "postgres",
    "password": "11223344",
    "driver": "org.postgresql.Driver"
}
pg_table = "employee_leave_data"

# SparkSession
spark = SparkSession.builder.appName("DailyLeaveSnapshot").getOrCreate()

# 1) LOAD raw events and cast date
raw = (
    spark.read
    .option("header", True)
    .csv(BRONZE_PATH)
    .withColumn("date", to_date(col("date"), "yyyy-M-d"))
)

# 2) AGGREGATE to find dominant status per emp/date
raw_agg = (
    raw.groupBy("emp_id", "date")
    .agg(
        sum_(when(col("status") == "ACTIVE", 1).otherwise(0)).alias("cnt_active"),
        sum_(when(col("status") == "CANCELLED", 1).otherwise(0)).alias("cnt_cancelled")
    )
    .withColumn(
        "new_status",
        when(col("cnt_active") > col("cnt_cancelled"), lit("ACTIVE"))
        .otherwise(lit("CANCELLED"))  # includes tie and CANCELLED majority cases
    )
    .select("emp_id", "date", "new_status")
)

# 3) READ previous snapshot (if any)
try:
    hist = (
        spark.read
        .parquet(CONSOLIDATED)
        .select("emp_id", "date", "status", "ingestion_date")
    )
except Exception as e:
    # If no history exists, create an empty DataFrame with the required schema.
    hist = spark.createDataFrame(
        [],
        schema=raw_agg.schema.add("status", StringType()).add("ingestion_date", DateType())
    )

# 4) MERGE logic: outer-join of aggregated (raw_agg) & history
merged = hist.alias("h").join(
    raw_agg.alias("r"),
    on=["emp_id", "date"],
    how="outer"
)

# 5) Decide final status and ingestion_date
today = current_date()
result = (
    merged.withColumn(
        "final_status",
        when(col("r.new_status").isNotNull(), col("r.new_status"))
        .otherwise(col("h.status"))
    )
    .withColumn(
        "ingestion_date",
        when(col("r.new_status").isNotNull(), today)
        .otherwise(col("h.ingestion_date"))
    )
    .filter(col("final_status").isNotNull())
    .select(
        col("emp_id"),
        col("date"),
        col("final_status").alias("status"),
        col("ingestion_date")
    )
)

# 6) Add partitions based on the leave date
result_with_partition = (
    result
    .withColumn("year", year(col("date")))
    .withColumn("month", month(col("date")))
)


# Broadcast join with Employee Timeframe data
employee_timeframe_path = "s3://poc-bootcamp-capstone-group3/poc-bootcamp-gold/employee_timeframe_data_output/"

# Load the employee timeframe data and filter to get only active employees.
employee_active = (
    spark.read.parquet(employee_timeframe_path)
    .filter(col("status") == "ACTIVE")
    .select("emp_id")
    .distinct()
    .withColumnRenamed("emp_id", "active_emp_id")
)

# Perform a broadcast join to check if each employee in the leave data is active.
# Then, update the leave status:
# - If the employee is NOT active (no match found), force the status to "CANCELLED".
# - Otherwise, retain the existing leave status from the aggregation logic.
result_with_partition = result_with_partition.join(
    broadcast(employee_active),
    result_with_partition.emp_id == col("active_emp_id"),
    how="left"
).withColumn(
    "status",
    when(col("active_emp_id").isNull(), lit("CANCELLED")).otherwise(col("status"))
).drop("active_emp_id")


# 7) Write the final result to Gold with partitions


(
    result_with_partition
    .write
    .mode("overwrite")
    .partitionBy("year", "month")
    .parquet(CONSOLIDATED)
)

#Write to Postgres
result_with_partition.write.mode("overwrite").jdbc(url=pg_url, table=pg_table, properties=pg_properties)



).withColumn(
    "status",
    when(col("active_emp_id").isNull(), lit("CANCELLED")).otherwise(col("status"))
).drop("active_emp_id")


# 7) Write the final result to Gold with partitions


(
    result_with_partition
    .write
    .mode("overwrite")
    .partitionBy("year", "month")
    .parquet(CONSOLIDATED)
)

#Write to Postgres
result_with_partition.write.mode("overwrite").jdbc(url=pg_url, table=pg_table, properties=pg_properties)




Task 5 - timeframe_data

In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from datetime import datetime
import boto3
import psycopg2

# Glue boilerplate
args = getResolvedOptions(sys.argv, ['JOB_NAME'])
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

# Config
config = {
    "pg_url": "jdbc:postgresql://54.174.233.120:5432/postgres",
    "pg_properties": {
        "user": "postgres",
        "password": "11223344",
        "driver": "org.postgresql.Driver"
    },
    "bucket_name": "poc-bootcamp-capstone-group3",
    "bronze_prefix": "poc-bootcamp-bronze/employee_timeframe_data/",
    "log_file_key": "poc-bootcamp-bronze/employee_timeframe_data/processed_files.txt",
    "silver_prefix": "poc-bootcamp-silver/employee_timeframe_data/"
}

today_date = datetime.today().strftime('%Y-%m-%d')
s3_client = boto3.client('s3')

# Step 1: Get list of processed files
def get_processed_files():
    try:
        response = s3_client.get_object(Bucket=config['bucket_name'], Key=config['log_file_key'])
        return {line.split(',')[0]: float(line.split(',')[1]) 
                for line in response['Body'].read().decode().splitlines() if line}
    except s3_client.exceptions.NoSuchKey:
        return {}

# Step 2: List new/modified files in bronze
def get_new_files():
    processed_files = get_processed_files()
    response = s3_client.list_objects_v2(Bucket=config['bucket_name'], Prefix=config['bronze_prefix'])
    new_files = []
    for obj in response.get('Contents', []):
        key = obj['Key']
        if key.endswith(".csv") and key != config['log_file_key']:
            modified_time = obj['LastModified'].timestamp()
            if key not in processed_files or processed_files[key] != modified_time:
                new_files.append(key)
    return new_files

# Step 3: Update log with newly processed files
def update_processed_files(new_files):
    processed_files = get_processed_files()
    for file in new_files:
        last_modified = s3_client.head_object(Bucket=config['bucket_name'], Key=file)['LastModified'].timestamp()
        processed_files[file] = last_modified
    log_content = "\n".join(f"{k},{v}" for k, v in processed_files.items())
    s3_client.put_object(Bucket=config['bucket_name'], Key=config['log_file_key'], Body=log_content)

# Step 4: Read and process new CSV files
new_files = get_new_files()
if not new_files:
    print("No new files to process.")
    sys.exit(0)

paths = [f"s3://{config['bucket_name']}/{key}" for key in new_files]
df = spark.read.option("header", "true").option("inferSchema", "true").csv(paths)

# Step 5: Transformations
df = df.withColumn("start_date", to_date(from_unixtime(col("start_date")))) \
        .withColumn("end_date", to_date(from_unixtime(col("end_date"))))

window_spec = Window.partitionBy("emp_id", "start_date", "end_date").orderBy(col("salary").desc())
df = df.withColumn("row_num", row_number().over(window_spec)).filter("row_num = 1").drop("row_num")

window_emp = Window.partitionBy("emp_id").orderBy("start_date")
df = df.withColumn("next_start_date", lead("start_date").over(window_emp))
df = df.withColumn("end_date", when(col("end_date").isNull(), col("next_start_date")).otherwise(col("end_date"))).drop("next_start_date")

df = df.withColumn("status", when(col("end_date").isNull(), "ACTIVE").otherwise("INACTIVE")) \
       .select("emp_id", "start_date", "end_date", "designation", "salary", "status")

# Step 6: Write to Silver Layer in S3
silver_path = f"s3://{config['bucket_name']}/{config['silver_prefix']}{today_date}/"
df.write.mode("append").parquet(silver_path)
print(f"Written processed data to Silver at {silver_path}")


db_config = {
    "host": "54.174.233.120",
    "port": "5432",
    "database": "postgres",
    "user": "postgres",
    "password": "11223344"
}
print("db_config done")



staging_table = "employee_time_staging"
main_table = "employee_timeframe"



min_id, max_id = df.selectExpr("min(emp_id) as min_id", "max(emp_id) as max_id").first()


df = df.persist()


try:
    conn = psycopg2.connect(**db_config)
    cur = conn.cursor()
    cur.execute(f"TRUNCATE TABLE {staging_table};")
    conn.commit()
except Exception as e:
    print(f"Truncate error: {e}")
finally:
    cur.close()
    conn.close()

# Step 1: Write to staging table
df.write \
    .format("jdbc") \
    .option("url", f"jdbc:postgresql://{db_config['host']}:{db_config['port']}/{db_config['database']}") \
    .option("dbtable", staging_table) \
    .option("user", db_config["user"]) \
    .option("password", db_config["password"]) \
    .option("driver", "org.postgresql.Driver") \
    .option("partitionColumn", "emp_id")\
    .option("lowerBound", min_id) \
    .option("upperBound", max_id) \
    .option("numPartitions", "8") \
    .mode("append") \
    .save()

print("Written data to staging table in PostgreSQL.")

# Step 2: Connect to PostgreSQL and upsert into main table
try:
    conn = psycopg2.connect(**db_config)
    cur = conn.cursor()
    
    # Create tables if they don't exist
    cur.execute(f"""
    CREATE TABLE IF NOT EXISTS {main_table} (
        emp_id BIGINT,
        start_date DATE,
        end_date DATE,
        designation TEXT,
        salary FLOAT,
        status TEXT
    );
    """)
    cur.execute(f"""
    CREATE TABLE IF NOT EXISTS {staging_table} (
        emp_id BIGINT,
        start_date DATE,
        end_date DATE,
        designation TEXT,
        salary FLOAT,
        status TEXT
    );
    """)

    # Upsert logic
    cur.execute(f"""
   -- Update existing ACTIVE records to set their end_date to the new record's start_date
UPDATE {main_table} m
SET end_date = s.start_date,
    status = 'INACTIVE'
FROM {staging_table} s
WHERE m.emp_id = s.emp_id
  AND m.status = 'ACTIVE'
  AND s.start_date > m.start_date;  -- Ensure the new record has a later start date than the existing one


    -- Step 2: Insert new records from staging
INSERT INTO {main_table} (emp_id, start_date, end_date, designation, salary, status)
SELECT s.emp_id, 
       s.start_date, 
       s.end_date,
       s.designation, 
       s.salary, 
       s.status
FROM {staging_table} s
    """)

    conn.commit()
except Exception as e:
    print(f"Error during DB operations: {e}")
    conn.rollback()
finally:
    cur.close()
    conn.close()
df.unpersist()

print("Data merged into employee_time table with correct ACTIVE/INACTIVE status.")

# Writing to gold

gold_df= spark.read \
    .format("jdbc") \
    .option("url", f"jdbc:postgresql://{db_config['host']}:{db_config['port']}/{db_config['database']}") \
    .option("dbtable", main_table) \
    .option("user", db_config["user"]) \
    .option("password", db_config["password"]) \
    .option("driver", "org.postgresql.Driver") \
    .load()

s3_output_path = "s3://poc-bootcamp-capstone-group3/poc-bootcamp-gold/employee_timeframe_data_output/"

gold_df.write.mode("overwrite").partitionBy("status").parquet(s3_output_path)

print(f"Data exported successfully to: {s3_output_path}")




# Step 9: Update log
update_processed_files(new_files)
print("Processed file log updated.")

job.commit()

Task 6 - Count by designation

In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.context import SparkContext
from pyspark.sql.functions import col, current_date
from datetime import datetime
import boto3

# PostgreSQL connection details
pg_url = "jdbc:postgresql://54.174.233.120:5432/postgres"
pg_properties = {
    "user": "postgres",
    "password": "11223344",
    "driver": "org.postgresql.Driver"
}
pg_table = "employee_designation"

# Initialize Spark and Glue contexts
args = getResolvedOptions(sys.argv, ['JOB_NAME'])
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

# Define S3 paths
gold_path = "s3://poc-bootcamp-capstone-group3/poc-bootcamp-gold/employee_timeframe_data_output/"
output_path = "s3://poc-bootcamp-capstone-group3/poc-bootcamp-gold/daily_active_employees_by_designation_output/"

# Read Gold Layer timeframe data
df = spark.read.parquet(gold_path)

# Filter records where today is between start_date and (end_date is null)
today = datetime.utcnow().date()
active_employees_df = df.filter(
    (col("start_date") <= today) &
    (col("end_date").isNull())
)

# Group by designation and count
summary_df = active_employees_df.groupBy("designation").count().withColumnRenamed("count", "active_count")

# Add snapshot date column for partitioning
summary_df = summary_df.withColumn("snapshot_date", current_date())

# Write to S3 partitioned by snapshot_date
summary_df.write.mode("append").partitionBy("snapshot_date").parquet(output_path)

print("Daily active employee snapshot generated and saved to S3.")

#Write to Postgres
summary_df.write.mode("overwrite").jdbc(url=pg_url, table=pg_table, properties=pg_properties)

# Commit job
job.commit()


Task 7 - 8% Threshold

In [ ]:
import datetime
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, countDistinct, dayofweek, lit
from pyspark.sql.types import DateType
from awsglue.context import GlueContext
from awsglue.dynamicframe import DynamicFrame
from pyspark.context import SparkContext
from awsglue.utils import getResolvedOptions

# PostgreSQL connection details
pg_url = "jdbc:postgresql://54.174.233.120:5432/postgres"
pg_properties = {
    "user": "postgres",
    "password": "11223344",
    "driver": "org.postgresql.Driver"
}
pg_table = "employee_ex"

# Initialize Spark & Glue Context
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session

# Get job parameters
args = getResolvedOptions(sys.argv, ['YEAR', 'TODAY_DATE'])

# PARAMETERS
YEAR = int(args['YEAR'])  # Convert YEAR to integer
THRESHOLD_PCT =0.08
today = datetime.datetime.strptime(args['TODAY_DATE'], '%Y-%m-%d').date()  # Convert string to date

# FILE PATHS
leaves_path = "s3://poc-bootcamp-capstone-group3/poc-bootcamp-gold/employee_leave_data_output/"
holidays_path = "s3://poc-bootcamp-capstone-group3/poc-bootcamp-bronze/employee_leave_calendar_data/"
http://54.174.233.120:3000/public-dashboards/c851c17993d4414db5a1eda9aabba085
# LOAD LEAVE DATA 
leaves_df = spark.read.parquet(leaves_path).withColumn("date", col("date").cast(DateType()))

# LOAD HOLIDAY DATA 
holidays_df = spark.read.option("header", True).csv(holidays_path).withColumn("date", col("date").cast(DateType()))

# BUILD CALENDAR (Excluding weekends and holidays)
start = today + datetime.timedelta(days=1)  # Start from tomorrow
end = datetime.date(YEAR, 12, 31)  # End on December 31

# Generate range of dates from tomorrow to Dec 31
days_df = spark.range(0, (end - start).days + 1).select((lit(start) + col("id").cast("int")).alias("date"))

# Filter out weekends and holidays
working_days_df = days_df.join(holidays_df, on="date", how="left_anti").filter(dayofweek(col("date")).between(2, 6))

total_working_days = working_days_df.count()
print("Total upcoming working days:", total_working_days)

# FILTER ACTIVE LEAVES ON FUTURE WORKING DAYS (Excluding cancelled)
active_leaves_df = (
    leaves_df.filter((col("status") == "ACTIVE") & (col("date") > lit(today)))
    .join(working_days_df, on="date", how="inner")
    .dropDuplicates(["emp_id", "date"])
)

# COUNT LEAVE APPLICATIONS PER EMPLOYEE
emp_leave_counts_df = active_leaves_df.groupBy("emp_id").agg(countDistinct("date").alias("upcoming_leaves"))

# APPLY 8% THRESHOLD
result_df = emp_leave_counts_df.filter(col("upcoming_leaves") > THRESHOLD_PCT * lit(total_working_days))

# Write output to S3 in Parquet format
output_path = "s3://poc-bootcamp-capstone-group3/poc-bootcamp-gold/8%-Threshold_output/"
result_df.write.parquet(output_path, mode="overwrite")

#Write to Postgres
result_df.write.mode("overwrite").jdbc(url=pg_url, table=pg_table, properties=pg_properties)


Task 8 - 80% Threshold

In [ ]:
import sys
import datetime
import time
import os
import boto3
from botocore.exceptions import ClientError
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    to_date, col, year, month, countDistinct, concat_ws, lit, date_format
)
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.utils import getResolvedOptions

# PostgreSQL connection details
pg_url = "jdbc:postgresql://54.174.233.120:5432/postgres"
pg_properties = {
    "user": "postgres",
    "password": "11223344",
    "driver": "org.postgresql.Driver"
}
pg_table = "employee_leaves_exceeding_80"

# Glue setup
args = getResolvedOptions(sys.argv, ['JOB_NAME'])
sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

# S3 Config
LEAVE_DATA_PATH     = "s3://poc-bootcamp-capstone-group3/poc-bootcamp-gold/employee_leave_data_output/"
LEAVE_CALENDAR_PATH = "s3://poc-bootcamp-capstone-group3/poc-bootcamp-bronze/employee_leave_calendar_data/"
LEAVE_QUOTA_PATH    = "s3://poc-bootcamp-capstone-group3/poc-bootcamp-bronze/employee_leave_quota/"

ALERT_TEXT_OUTPUT_PATH   = "s3://poc-bootcamp-capstone-group3/poc-bootcamp-gold/employee_80%_text_output/"
PARQUET_OUTPUT_PATH      = "s3://poc-bootcamp-capstone-group3/poc-bootcamp-gold/employee_80%_parquet_output/"
METADATA_KEY             = "poc-bootcamp-bronze/80%Threshold/metadata.txt"
ALERTS_BUCKET            = "poc-bootcamp-capstone-group3"

# Reference date and reporting period
ref_date = datetime.date(2024, 11, 1)
# ref_date = datetime.date.today()

report_month = ref_date.month - 1 or 12
report_year = ref_date.year if ref_date.month > 1 else ref_date.year - 1
period = f"{report_year}-{report_month:02d}"

# Load previously processed metadata
s3 = boto3.client("s3")
processed = set()
tmp_meta = "/tmp/metadata.txt"
try:
    s3.download_file(ALERTS_BUCKET, METADATA_KEY, tmp_meta)
    with open(tmp_meta, "r") as f:
        processed = set(line.strip() for line in f)
except ClientError as e:
    if e.response['Error']['Code'] != 'NoSuchKey':
        raise

# Load and clean data
leave_df = (
    spark.read.parquet(LEAVE_DATA_PATH)
         .withColumn("date", to_date("date", "yyyy-M-d"))
         .filter(col("status") == "ACTIVE")
         .dropDuplicates(["emp_id", "date"])
)

holidays_df = (
    spark.read.option("header", True).csv(LEAVE_CALENDAR_PATH)
         .withColumn("date", to_date("date", "yyyy-M-d"))
         .select("date").distinct()
)

clean_leaves = (
    leave_df.withColumn("dow", date_format("date", "E"))
            .filter(~col("dow").isin("Sat", "Sun"))
            .drop("dow")
            .join(holidays_df, on="date", how="left_anti")
)

quota_df = spark.read.option("header", True).csv(LEAVE_QUOTA_PATH)

# Filter only leaves in the current reporting year up to the reporting month
up_to = clean_leaves.filter(
    (year("date") == report_year) &
    (month("date") <= report_month)
)

# Count leave days per employee
counts_df = up_to.groupBy("emp_id").agg(countDistinct("date").alias("used"))

# Compare with quota and find violators
breachers_df = (
    counts_df.join(quota_df, on="emp_id", how="inner")
             .filter((col("used") / col("leave_quota")) > 0.8)
             .select("emp_id")
)

# Avoid duplicates
to_alert = [
    emp for emp in breachers_df.collect()
    if f"{period},{emp.emp_id}" not in processed
]

# Process alerts if any
if to_alert:
    ts = time.strftime("%Y%m%d-%H%M%S")
    text_out_path = ALERT_TEXT_OUTPUT_PATH 

    lines_df = spark.createDataFrame(
        [(e.emp_id, period) for e in to_alert],
        ["emp_id", "month"]
    ).select(concat_ws(",", "emp_id", "month").alias("line"))

    lines_df.write.mode("append").text(text_out_path)
    print(f"Alerted {len(to_alert)} employees → {text_out_path}")

    # Write Parquet of employees who breached the limit with their used leave count
    breachers_with_count_df = (
        breachers_df.join(counts_df, on="emp_id", how="inner")
                    .select(
                        col("emp_id"),
                        col("used").alias("count_of_leaves"),
                        lit(report_year).alias("year"),
                        lit(report_month).alias("month")
                    )
                    .filter(col("emp_id").isin([e.emp_id for e in to_alert]))
    )

    parquet_out_path = PARQUET_OUTPUT_PATH 
    breachers_with_count_df.write.partitionBy("year", "month").mode("append").parquet(parquet_out_path)
    print(f"Parquet file written with leave counts → {parquet_out_path}")

    #Write to Postgres
    breachers_with_count_df.write.mode("append").jdbc(url=pg_url, table=pg_table, properties=pg_properties)

    # ── 11) Update metadata
    with open(tmp_meta, "a") as f:
        for emp in to_alert:
            f.write(f"{period},{emp.emp_id}\n")
    s3.upload_file(tmp_meta, ALERTS_BUCKET, METADATA_KEY)
    os.remove(tmp_meta)
else:
    print(f"No new alerts for {period}")


job.commit()
